# Word2Vec - Treinamento de word embbeding

### Bibliotecas básicas e outras inicializações

In [1]:
import pandas as pd
import numpy as np
import spacy
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
%matplotlib inline

### Carregando os dados

In [2]:
dados_treino = pd.read_csv('./datasets/treino.csv')
print(dados_treino.shape)
dados_treino.sample()

(90000, 6)


,title,text,date,category,subcategory,link
76384,Duas jovens são sequestradas e sofrem estupro ...,"Duas adolescentes, de 14 e 15 anos, foram sequ...",2016-01-10,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/10...


In [3]:
dados_teste = pd.read_csv('./datasets/teste.csv')
print(dados_teste.shape)
dados_teste.sample()

(20513, 6)


,title,text,date,category,subcategory,link
12685,Real Madrid é campeão de Supercopa da Uefa com...,O Real Madrid começou a temporada 2017-2018 co...,2017-08-08,esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/08/1...


### Pré-processamento do dados

In [4]:
nlp = spacy.load('pt_core_news_sm')